In [ ]:
import win32com.client as win32
import os
import logging
import pandas as pd

def 엑셀열기():
    file_path_excel = r"C:\Users\thdus\OneDrive\문서\서식34.xlsx" # 엑셀 파일 경로 지정
    sheet_name = "참여자 명단"  # 엑셀 시트 이름
    excel_data = pd.read_excel(file_path_excel, sheet_name=sheet_name, header=1, dtype=str).fillna("")
    
    return excel_data

# 엑셀 데이터를 불러옵니다.
excel_data = 엑셀열기()
print(f"{excel_data['성명']}")

hwp = win32.gencache.EnsureDispatch("hwpframe.hwpobject")
hwp.XHwpWindows.Item(0).Visible = True  # 한글 창을 보이게 설정
hwp.RegisterModule("FilePathCheckDLL", "FilePathCheckerModuleExample")

def 파일열기(hwp, file_name, file_path, file_확장자):
    # 전체 경로를 직접 결합
    full_path= os.path.join(file_path, file_name+file_확장자)
    return hwp.Open(full_path)

file_name = '[서식33] 인턴형 일경험 참여신청서'
file_확장자= '.hwp'
file_path = r"C:\Users\thdus\Desktop\파이썬 + 한글_자동화"
full_path = 파일열기(hwp, file_name, file_path, file_확장자)

def insert_signature_image(hwp, image_path, target_field_name, width, height, x_pos, y_pos):
    try:
        # 서명 필드로 커서 이동
        hwp.MoveToField(target_field_name)

        # 이미지 삽입하기
        hwp.InsertPicture(image_path, Embedded=True, sizeoption=0)
        
        # 삽입된 이미지 선택
        hwp.FindCtrl()  # 개체를 선택해야 이후 설정 적용 가능
        
        # 설정 변경
        hwp.HAction.GetDefault("ShapeObjDialog", hwp.HParameterSet.HShapeObject.HSet)
        hwp.HParameterSet.HShapeObject.HSet.SetItem("Height", int(height * 283.465))  # 높이 설정
        hwp.HParameterSet.HShapeObject.HSet.SetItem("Width", int(width * 283.465))   # 너비 설정
        hwp.HParameterSet.HShapeObject.HSet.SetItem("TreatAsChar", 0)  # 글자처럼 취급 해제 설정
        hwp.HParameterSet.HShapeObject.HSet.SetItem("TextWrap", 3)  # 글 앞에 배치
        hwp.HParameterSet.HShapeObject.HSet.SetItem("HorzRelTo", 0)  # 속성을 종이 기준으로 설정
        hwp.HParameterSet.HShapeObject.HSet.SetItem("VertRelTo", 0)  # 속성을 종이 기준으로 설정
        hwp.HParameterSet.HShapeObject.HSet.SetItem("HorzOffset", hwp.MiliToHwpUnit(x_pos))
        hwp.HParameterSet.HShapeObject.HSet.SetItem("VertOffset", hwp.MiliToHwpUnit(y_pos))
        
        # 설정 적용
        hwp.HAction.Execute("ShapeObjDialog", hwp.HParameterSet.HShapeObject.HSet)

        print(f"서명 이미지가 '{target_field_name}' 필드에 삽입되었습니다.")
    except Exception as e:
        print(f"Error inserting image at {target_field_name}: {e}")

# 엑셀 데이터를 한글 파일에 집어넣기
for index, row in excel_data.iterrows():
    # 매 행마다 한글 파일을 다시 열기
    파일열기(hwp, file_name, file_path, file_확장자)
    

    # 성명이 비어있다면 해당 행을 건너뜀
    if not row['성명']:  # 성명 값이 없는 경우 현재 행을 건너뜀
        print(f"빈 값 발견 - 행 {index + 1} 건너뜀")
        break

    try:
        # 필드에 데이터 삽입
        hwp.PutFieldText('성명', row['성명'])
        hwp.PutFieldText('주민등록번호', row['주민등록번호'])
        hwp.PutFieldText('만나이', row['만나이'])
        hwp.PutFieldText('주소', row['주소'])
        hwp.PutFieldText('연락처', row['연락처'])
        hwp.PutFieldText('이메일', row['이메일'])    
        hwp.PutFieldText('작성날짜', row['작성날짜'])
        
        # 최종학력구분에 따라 기호를 추가 (대학생이면 O 표시, 고등학생이면 X 표시 등)
        if row['최종학력구분'] == '대학교_4년제':
            hwp.PutFieldText('대학교', '■')
            hwp.PutFieldText('대학교명', row['최종학교명'])
            hwp.PutFieldText('최종학과', row['최종학과'])
            if row['최종학력상태'] == '졸업':
                hwp.PutFieldText('대학교졸업', '■')

            elif row['최종학력상태'] == '졸업예정자':
                hwp.PutFieldText('졸업예정자', '■')

            elif row['최종학력상태'] == '휴학':
                hwp.PutFieldText('휴학', '■')
                  

            elif row['최종학력상태'] == '재학':
                if row['학년'] in ['1','2'] :
                    hwp.PutFieldText('재학(1~2학년)', '■')
                elif row['학년'] in ['3','4']:
                    hwp.PutFieldText('재학(3~4학년)', '■')   



        # 대학교_2/3년제
        if row['최종학력구분'] == '대학교_2/3년제':
            hwp.PutFieldText('대학교', '■')
            hwp.PutFieldText('대학교명', row['최종학교명'])
            hwp.PutFieldText('최종학과', row['최종학과'])
            if row['최종학력상태'] == '졸업':
                hwp.PutFieldText('대학교졸업', '■')

            elif row['최종학력상태'] == '졸업예정자':
                hwp.PutFieldText('졸업예정자', '■')

            elif row['최종학력상태'] == '휴학':
                if row['학년'] in ['1','2'] :
                    hwp.PutFieldText('재학(1~2학년)', '■')
                elif row['학년'] in ['3','4']:
                    hwp.PutFieldText('재학(3~4학년)', '■')       

            elif row['최종학력상태'] == '재학':
                if row['학년'] in ['1','2'] :
                    hwp.PutFieldText('재학(1~2학년)', '■')
                elif row['학년'] in ['3']:
                    hwp.PutFieldText('재학(3~4학년)', '■')

        # 대학교_2년제
        if row['최종학력구분'] == '대학교_2년제':
            hwp.PutFieldText('대학교', '■')
            hwp.PutFieldText('대학교명', row['최종학교명'])
            hwp.PutFieldText('최종학과', row['최종학과'])
            if row['최종학력상태'] == '졸업':
                hwp.PutFieldText('대학교졸업', '■')

            elif row['최종학력상태'] == '졸업예정자':
                hwp.PutFieldText('졸업예정자', '■')

            elif row['최종학력상태'] == '휴학':
                if row['학년'] in ['1','2'] :
                    hwp.PutFieldText('재학(1~2학년)', '■')
                      
            elif row['최종학력상태'] == '재학':
                if row['학년'] in ['1','2'] :
                    hwp.PutFieldText('재학(1~2학년)', '■')
                 


        # 고등학교
        if row['최종학력구분'] == '고등학교':
            hwp.PutFieldText('고등학교', '■')
            hwp.PutFieldText('고등학교명', row['최종학교명'])
            if row['최종학력상태'] == '졸업':
                hwp.PutFieldText('고등학교졸업', '■')
                 
            elif row['최종학력상태'] == '검정고시':
                hwp.PutFieldText('검정고시', '■')

            elif row['최종학력상태'] == '재학':
                if row['학년'] in ['1','2'] :
                    hwp.PutFieldText('재학(1~2학년)', '■')
                elif row['학년'] in ['3','4']:
                    hwp.PutFieldText('재학(3~4학년)', '■')


        # 서명 이미지 삽입
        if os.path.exists(image_path):
            insert_signature_image(hwp, image_path, "서명", 19, 6, 112.56, 232.49)
        else:
            print(f"서명 이미지가 존재하지 않습니다: {image_path}")


        # 새로운 파일 이름 생성
        new_file_name = f"{file_name}_{row['성명']}.hwp"
        new_file_path = os.path.join(file_path, new_file_name)

        # 파일 다른 이름으로 저장
        hwp.SaveAs(new_file_path)
        logging.info(f"파일 저장 완료: {new_file_name}")
        hwp.Clear(1)

    except Exception as e:
        logging.error(f"오류 발생 (행 {index+1}): {e}")

# 엑셀 반복이 종료되면 한글 종료
hwp.Quit()
os.startfile(file_path)

0    정민주
1    김성한
2    이소연
3    김주혁
Name: 성명, dtype: object


In [64]:

hwp = win32.gencache.EnsureDispatch("hwpframe.hwpobject")
hwp.XHwpWindows.Item(0).Visible = True  # 한글 창을 보이게 설정
hwp.RegisterModule("FilePathCheckDLL", "FilePathCheckerModuleExample")

def 파일열기(hwp, file_name, file_path, file_확장자):
    # 전체 경로를 직접 결합
    full_path= os.path.join(file_path, file_name+file_확장자)
    return hwp.Open(full_path)

file_name = '[서식33] 인턴형 일경험 참여신청서'
file_확장자= '.hwp'
file_path = r"C:\Users\thdus\Desktop\파이썬 + 한글_자동화"
full_path = 파일열기(hwp, file_name, file_path, file_확장자)
image_path= r'C:\Users\thdus\Desktop\파이썬 + 한글_자동화\서명\김성한.png'
# 필드로 커서 이동

hwp.MoveToField("서명")  # 테이블이 필드 안에 있는 경우

# 이미지 삽입
hwp.InsertPicture(image_path)


<win32com.gen_py.HwpObject 1.0 Type Library.IDHwpCtrlCode instance at 0x2667601022016>